# <center> <img src="img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
### <center> **Spring 2025** </center>
---

**Mid-term exam**

**Date**: 3/12/2025

**Student Name**:

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Exam-1-ADD-HERE-YOUR-NAME") \
    .master("spark://078b2e28e517:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/12 03:19:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Load datasets

In [4]:
from team_name.spark_utils import SparkUtils

column_info_sales = [("transaction_id", "integer"), ("product_id", "string"), ("customer_id", "integer"), ("sale_ammount", "double"), ("sale_date", "date")]
sales_schema = SparkUtils.generate_schema(column_info_sales)

sales_df = spark.read \
                .schema(sales_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/exam_P2025_ESI3914N/sales.csv")

sales_df.printSchema()

root
 |-- transaction_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- sale_ammount: double (nullable = true)
 |-- sale_date: date (nullable = true)



In [5]:

column_info_product = [("product_id", "integer"), ("product_name", "string"), ("category", "string"), ("price", "double")]
product_schema = SparkUtils.generate_schema(column_info_product)

product_df = spark.read \
                .schema(product_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/exam_P2025_ESI3914N/products.csv")

product_df.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: double (nullable = true)



In [14]:
from pyspark.sql.functions import col, sum, when
# 2. Join the DataFrames
enriched_sales_df = sales_df.join(product_df, on="product_id", how="inner")

# 3. Group and Aggregate
category_sales_df = enriched_sales_df.groupBy("category").agg(sum("sale_ammount").alias("total_sales"))

# 5. Transformations (Using `when()`)
enriched_sales_df = enriched_sales_df.withColumn(
    "sale_type",
    when(col("sale_ammount") >= 50, "High").otherwise("Low")
)

# 6. Filter
electronics_sales_df = enriched_sales_df.filter(col("category") == "Electronics")

# 7. Select
final_df = electronics_sales_df.select("product_name", "sale_ammount", "sale_type")

# 8. Action (Collect and Display)
final_df.count()

612